In [1]:
from ast import literal_eval
import os
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# constants and global variables
PROBABILITIES = '/content/drive/MyDrive/Colab Notebooks/CIL/Probabilities/{}'
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/CIL/Dataset/{}'
PREDICTIONS = '/content/drive/MyDrive/Colab Notebooks/CIL/Predictions/{}'

In [54]:
# Perform a voting --> use the probabilities of the predictions from different
# models to create a new set of predictions
y_pred = []
for filename in os.listdir('/content/drive/MyDrive/Colab Notebooks/CIL/Probabilities/'):
  # open file containing probabilities of a model
  print(filename)
  f = open(PROBABILITIES.format(filename), "r")

  # discard header
  header = f.readline()

  # read the enitire file
  probas = f.readlines()

  to_sum = []
  # loop over prediction probabilties for each X_test
  for proba in probas:
      # discard id and \n
      proba = ",".join(proba.split(",")[1:]).strip()
      # go from string to array
      proba = literal_eval(proba)

      to_sum.append(np.array(proba))
  
  if y_pred == []:
    y_pred = to_sum
  else: 
    y_pred = np.add(y_pred, to_sum)

bert-base-cased_predictions_probas.csv
cardiffnlp_twitter-roberta-base-sentiment-latest_predictions_probas.csv


In [55]:
y_pred = np.array(np.argmax(np.array(y_pred), axis=-1))

def write_output(file_name, Y):
  f  = open(file_name, "w")
  f.write("Id,Prediction\n")
  id = 1
  for y in Y:
    if y == 0:
      y = -1
    f.write(str(id) + "," + str(y) + "\n")
    id = id + 1
  f.close()

write_output(PREDICTIONS.format("ensemble_predictions.csv"), y_pred)